# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [ ]:
import random

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
    
    def value(self):
        # Return the blackjack value of the card
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11  # Ace is initially counted as 11
        else:
            return int(self.rank)

    def __str__(self):
        return f"{self.rank} of {self.suit}"


class Deck:
    def __init__(self, num_decks=6):
        self.num_decks = num_decks
        self.cards = []
        self._initialize_deck()
    
    def _initialize_deck(self):
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for suit in suits for rank in ranks] * self.num_decks
        random.shuffle(self.cards)

    def draw_card(self):
        if not self.cards:
            self._initialize_deck()  # Reinitialize if deck runs out
        return self.cards.pop()

    def place_plastic_card(self):
        # Insert a "plastic" card to signify reshuffling
        self.cards.insert(random.randint(0, len(self.cards) - 1), "PLASTIC")

    def shuffle(self):
        random.shuffle(self.cards)


In [ ]:
class Player:
    def __init__(self, name, chips=100):
        self.name = name
        self.chips = chips
        self.hand = []
        self.bet = 0
    
    def place_bet(self, amount):
        self.bet = amount
        self.chips -= amount

    def receive_card(self, card):
        self.hand.append(card)

    def calculate_hand_value(self):
        value = sum(card.value() for card in self.hand if isinstance(card, Card))
        # Adjust for Aces if needed to avoid bust
        num_aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and num_aces:
            value -= 10
            num_aces -= 1
        return value

    def clear_hand(self):
        self.hand = []
        self.bet = 0

    def __str__(self):
        return f"{self.name} with chips: {self.chips}"


class Dealer(Player):
    def __init__(self):
        super().__init__(name="Dealer")
    
    def play_turn(self, deck):
        while self.calculate_hand_value() < 17:
            card = deck.draw_card()
            self.receive_card(card)



In [ ]:
class BlackjackGame:
    def __init__(self, num_decks=6, players=[]):
        self.deck = Deck(num_decks)
        self.players = players
        self.dealer = Dealer()

    def start_round(self):
        # Clear all hands and bets
        for player in self.players:
            player.clear_hand()
        self.dealer.clear_hand()
        
        # Deal initial two cards to each player and dealer
        for _ in range(2):
            for player in self.players:
                player.receive_card(self.deck.draw_card())
            self.dealer.receive_card(self.deck.draw_card())

    def player_turns(self):
        for player in self.players:
            while player.calculate_hand_value() < 21:
                action = player.decide_action()  # Implement decide_action for strategies
                if action == "hit":
                    player.receive_card(self.deck.draw_card())
                else:
                    break
    
    def dealer_turn(self):
        self.dealer.play_turn(self.deck)

    def determine_outcomes(self):
        dealer_value = self.dealer.calculate_hand_value()
        for player in self.players:
            player_value = player.calculate_hand_value()
            if player_value > 21:
                outcome = "lose"
            elif dealer_value > 21 or player_value > dealer_value:
                outcome = "win"
            elif player_value < dealer_value:
                outcome = "lose"
            else:
                outcome = "tie"
            # Update chips based on outcome (e.g., win doubles the bet)
            self.resolve_bet(player, outcome)

    def resolve_bet(self, player, outcome):
        if outcome == "win":
            player.chips += 2 * player.bet
        elif outcome == "tie":
            player.chips += player.bet  # Return bet on tie
        # No chips returned if player loses

    def play_game(self, num_rounds):
        for _ in range(num_rounds):
            self.start_round()
            self.player_turns()
            self.dealer_turn()
            self.determine_outcomes()
            # Reset deck if plastic card appears
            if "PLASTIC" in self.deck.cards:
                self.deck.shuffle()
                self.deck.place_plastic_card()



In [ ]:
class StrategyPlayer(Player):
    def __init__(self, name, chips=100, threshold=0):
        super().__init__(name, chips)
        self.threshold = threshold
        self.card_count = 0

    def update_card_count(self, card):
        if isinstance(card, Card):
            if card.rank in ['2', '3', '4', '5', '6']:
                self.card_count += 1
            elif card.rank in ['10', 'J', 'Q', 'K', 'A']:
                self.card_count -= 1

    def decide_action(self):
        return "hit" if self.card_count < self.threshold else "stay"

